In [2]:
import pandas as pd

# Load the provided CSV file to examine its structure
file_path = '../problem2/predicted_future_volumes_hours.csv'
predicted_volumes = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
predicted_volumes.head(), predicted_volumes.columns

(       分拣中心              日期时间       货量
 0  分拣中心_SC1  2023/11/01 00:00  5679.73
 1  分拣中心_SC1  2023/11/01 01:00  5130.17
 2  分拣中心_SC1  2023/11/01 02:00  5216.39
 3  分拣中心_SC1  2023/11/01 03:00  5019.86
 4  分拣中心_SC1  2023/11/01 04:00  4220.94,
 Index(['分拣中心', '日期时间', '货量'], dtype='object'))

In [3]:
# Convert the date-time string to a datetime object
predicted_volumes['日期时间'] = pd.to_datetime(predicted_volumes['日期时间'])

# Define the shift times
shift_times = {
    '00:00-08:00': ('00:00', '08:00'),
    '05:00-13:00': ('05:00', '13:00'),
    '08:00-16:00': ('08:00', '16:00'),
    '12:00-20:00': ('12:00', '20:00'),
    '14:00-22:00': ('14:00', '22:00'),
    '16:00-24:00': ('16:00', '24:00'),
}

# Create a function to assign each hour to a shift
def assign_shift(hour):
    for shift, (start, end) in shift_times.items():
        if start <= hour.strftime('%H:%M') < end:
            return shift
    return None

# Apply the function to assign shifts
predicted_volumes['班次'] = predicted_volumes['日期时间'].apply(lambda x: assign_shift(x))

# Group by center, date, and shift to sum up volumes
grouped_volumes = predicted_volumes.groupby(['分拣中心', predicted_volumes['日期时间'].dt.date, '班次'])['货量'].sum().reset_index()
grouped_volumes.rename(columns={'日期时间': '日期'}, inplace=True)

grouped_volumes.head()

,分拣中心,日期,班次,货量
0,分拣中心_SC1,2023-11-01,00:00-08:00,34294.59
1,分拣中心_SC1,2023-11-01,05:00-13:00,21413.50
2,分拣中心_SC1,2023-11-01,08:00-16:00,11589.62
3,分拣中心_SC1,2023-11-01,12:00-20:00,15372.30
4,分拣中心_SC1,2023-11-01,14:00-22:00,8951.26


In [4]:
import pulp
from pulp import PULP_CBC_CMD

# Set up the linear programming problem to minimize the total number of person-days
model = pulp.LpProblem("Staff_Scheduling", pulp.LpMinimize)

# Decision variables
# Number of regular and temporary workers per shift, per day, per sorting center
regular_workers = pulp.LpVariable.dicts(
    "Regular",
    [
        (center, date, shift)
        for center, date, shift in zip(
            grouped_volumes["分拣中心"],
            grouped_volumes["日期"],
            grouped_volumes["班次"],
        )
    ],
    lowBound=0,
    cat="Integer",
)
temporary_workers = pulp.LpVariable.dicts(
    "Temporary",
    [
        (center, date, shift)
        for center, date, shift in zip(
            grouped_volumes["分拣中心"],
            grouped_volumes["日期"],
            grouped_volumes["班次"],
        )
    ],
    lowBound=0,
    cat="Integer",
)

# Objective: Minimize the total person-days
model += pulp.lpSum(
    [
        regular_workers[center, date, shift] + temporary_workers[center, date, shift]
        for center, date, shift in zip(
            grouped_volumes["分拣中心"],
            grouped_volumes["日期"],
            grouped_volumes["班次"],
        )
    ]
)

In [5]:
# Constraints
# Each shift's staffing needs must meet the volume requirements
for i, row in grouped_volumes.iterrows():
    center, date, shift, volume = row["分拣中心"], row["日期"], row["班次"], row["货量"]
    model += (
        25 * regular_workers[center, date, shift]
        + 20 * temporary_workers[center, date, shift]
        >= volume
    )

# Maximum of 60 regular workers per sorting center per day
for (center, date), group in grouped_volumes.groupby(["分拣中心", "日期"]):
    model += (
        pulp.lpSum(regular_workers[center, date, shift] for shift in group["班次"])
        <= 60
    )

In [6]:
# Solve the model
model.solve(PULP_CBC_CMD(msg=1, threads=8, gapRel=1))
# model.solve()

KeyboardInterrupt: 

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/yinmo19/Mathor_cup_2024/venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/3d82e56b19fe4354ad3dae7f558d3ee8-pulp.mps -ratio 1 -threads 8 -timeMode elapsed -branch -printingOptions all -solution /tmp/3d82e56b19fe4354ad3dae7f558d3ee8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12374 COLUMNS
At line 107793 RHS
At line 120163 BOUNDS
At line 141368 ENDATA
Problem MODEL has 12369 rows, 21204 columns and 31806 elements
Coin0008I MODEL read with 0 errors
ratioGap was changed from 0 to 1
threads was changed from 0 to 8
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.35403e+06 - 0.14 seconds
Cgl0003I 0 fixed, 10602 tightened bounds, 121 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 95 strengthened rows, 0 substitutions
Cgl0004I processed model has 12369 rows, 21204 columns (21204 integ

In [ ]:

# Output results
results = []
for center, date, shift in zip(
    grouped_volumes["分拣中心"], grouped_volumes["日期"], grouped_volumes["班次"]
):
    result = {
        "分拣中心": center,
        "日期": date,
        "班次": shift,
        "正式工": regular_workers[center, date, shift].varValue,
        "临时工": temporary_workers[center, date, shift].varValue,
    }
    results.append(result)

results_df = pd.DataFrame(results)
results_df.head()